In [ ]:
import os
import math
import tensorflow as tf
import random
import numpy as np
import matplotlib.pyplot as plt
import datetime
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D, Dropout, Input, BatchNormalization,ReLU
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
import seaborn as sns
from tensorflow import keras
import pandas as pd

# Alexnet Architecture

In [ ]:
@tf.keras.utils.register_keras_serializable()
class AlexNet(Sequential):
    def __init__(self, input_shape, nb_classes, optimizer, l2 = 0.000005):
        super(AlexNet, self).__init__()
        self.add(Input(shape=input_shape)),
        self.add(Conv2D(96, kernel_size=(11,11), strides=(4,4), padding='valid', kernel_regularizer=keras.regularizers.l2(l2)))
        self.add(BatchNormalization())
        self.add(ReLU())
        self.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
        
        self.add(Conv2D(256, kernel_size=(5,5), strides=(1,1), padding='same', kernel_regularizer=keras.regularizers.l2(0.01)))
        self.add(BatchNormalization())
        self.add(ReLU())
        self.add(MaxPooling2D(pool_size=(3,3), strides=(2,2), padding='valid'))
        
        self.add(Conv2D(384, kernel_size=(3,3), strides=(1,1), padding='same', kernel_regularizer=keras.regularizers.l2(l2)))
        self.add(BatchNormalization())
        self.add(ReLU())
        
        self.add(Conv2D(384, kernel_size=(3,3), strides=(1,1), padding='same', kernel_regularizer=keras.regularizers.l2(l2)))
        self.add(BatchNormalization())
        self.add(ReLU())
        
        self.add(Conv2D(256, kernel_size=(3,3), strides=(1,1), padding='same',kernel_regularizer=keras.regularizers.l2(l2)))
        self.add(BatchNormalization())
        self.add(ReLU())
        
        self.add(Flatten())
        
        self.add(Dense(4096, kernel_regularizer=keras.regularizers.l2(l2)))
        self.add(BatchNormalization())
        self.add(ReLU())
        self.add(Dropout(0.5))
        
        self.add(Dense(4096, kernel_regularizer=keras.regularizers.l2(l2)))
        self.add(BatchNormalization())
        self.add(ReLU())
        self.add(Dropout(0.5))
        
        self.add(Dense(nb_classes, activation='softmax'))
        
        self.compile(optimizer = optimizer, loss = 'categorical_crossentropy', metrics = ['accuracy'])
        self.summary()

# Create a wrapper to train model

In [ ]:
class Model:
    def __init__(self, model, batch_size):
        self.model = model
        self.batch_size = batch_size
        
    def train(self,train_data, callbacks, epochs = 50):
        history = self.model.fit(
              train_data,
              steps_per_epoch= train_data.samples // self.batch_size,
              validation_data=validation_generator,
              validation_steps=validation_generator.samples // self.batch_size,
              callbacks = callbacks,
              epochs= epochs)
        
        return history;
        
    def evaluate(self, data):
        loss, acc = self.model.evaluate(data)
        print(f"acc: {acc}, loss: {loss}")

# Dataset Directory 

In [ ]:
train_data_dir = '/kaggle/input/intel-image-classification/seg_train/seg_train'
test_data_dir = '/kaggle/input/intel-image-classification/seg_test/seg_test'

# Create train and validation dataset

In [ ]:
img_size = (150, 150)
batch_size = 64

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255, 
    validation_split=0.2,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
)

train_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size= img_size,
    batch_size= batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='training',
)

validation_generator = datagen.flow_from_directory(
    train_data_dir,
    target_size= img_size,
    batch_size= batch_size,
    class_mode='categorical',
    shuffle=True,
    subset='validation'
)

# Create test dataset

In [ ]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size= img_size,
    batch_size= batch_size,
    class_mode='categorical',
    shuffle=False,
)

# Randomly Display Train and Validation Images

In [ ]:
def display_images_per_category(generator, categories, images_per_category=3):
    # Dictionary to store images for each category
    category_images = {category: [] for category in categories}
    
    # Iterate over the dataset to fetch images
    for images, labels in generator:
        for i, label in enumerate(labels):
            category = categories[np.argmax(label)]
            if len(category_images[category]) < images_per_category:
                category_images[category].append(images[i])
        
        # Stop when we have enough images for each category
        if all(len(imgs) >= images_per_category for imgs in category_images.values()):
            break

    # Plot the images
    num_categories = len(categories)
    plt.figure(figsize=(num_categories * 2, images_per_category * 2))
    
    for col_idx, category in enumerate(categories):
        for row_idx in range(images_per_category):
            # Only plot if there are enough images
            if row_idx < len(category_images[category]):
                plt.subplot(images_per_category, num_categories, row_idx * num_categories + col_idx + 1)
                plt.imshow(category_images[category][row_idx])
                plt.axis('off')
                
                # Show category name at the top of the first row
                if row_idx == 0:
                    plt.title(category)
    
    plt.tight_layout()
    plt.show()

In [ ]:
category_names = list(train_generator.class_indices.keys())
display_images_per_category(train_generator,category_names,images_per_category = 3)

In [ ]:
category_names = list(validation_generator.class_indices.keys())
display_images_per_category(validation_generator,category_names,images_per_category = 3)

In [ ]:
input_shape = train_generator.image_shape
category_count = len(category_names)

In [ ]:
class LearningRateLogger(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        current_lr = tf.keras.backend.get_value(self.model.optimizer.learning_rate)
        print(f" Learning Rate = {current_lr:.6f}")

In [ ]:
epochs = 70
initial_learning_rate = 3e-4
end_learning_rate = 9e-5
decay_steps = math.ceil((train_generator.samples // batch_size) * 1.2)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate = initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=0.95)

lr_logger = LearningRateLogger()

# lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
#     initial_learning_rate,
#     decay_steps,
#     end_learning_rate,
#     power=2.0)

adam = tf.keras.optimizers.Adam(learning_rate = lr_schedule)

In [ ]:
alex_net = AlexNet(input_shape, category_count, optimizer = adam)

# Saving checkpoint callback

In [ ]:
checkpoint_path = "/kaggle/working/training_1/cp.weights.h5"
checkpoint_dir = os.path.dirname(checkpoint_path)
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 monitor='val_loss',
                                                 mode='min',
                                                 save_best_only=True,
                                                 verbose=1)

# Train Model

In [ ]:
model = Model(alex_net, batch_size)
history = model.train(train_generator, epochs = epochs, callbacks = [lr_logger,cp_callback])

# Plotting Model Accuracy

In [ ]:
def plot_accuracy_values(history):
        plt.plot(history.history['accuracy'])
        plt.plot(history.history['val_accuracy'])
        plt.title('Model accuracy')
        plt.ylabel('Accuracy')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.show()

        # Plot training & validation loss values
        plt.plot(history.history['loss'])
        plt.plot(history.history['val_loss'])
        plt.title('Model loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plt.legend(['Train', 'Validation'], loc='upper left')
        plt.show()

In [ ]:
plot_accuracy_values(history)

# Evaluating Test Data

In [ ]:
if os.path.exists(checkpoint_path):
    model.model.load_weights(checkpoint_path)

In [ ]:
test_loss, test_acc = model.model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print(f'Test accuracy: {test_acc}')
print(f'Test loss: {test_loss}')

In [ ]:
predictions = model.model.predict(test_generator, steps=test_generator.samples // batch_size)
predicted_classes = predictions.argmax(axis=-1)

In [ ]:


predictions = model.model.predict(test_generator, steps=len(test_generator))
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_generator.classes

cm = confusion_matrix(true_classes, predicted_classes)

class_labels = list(test_generator.class_indices.keys())

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_labels, yticklabels=class_labels)
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()